## _Computer Vision_
### Lab 3, _Morphological Operations_.
#### >> Topics  :
* ##### Sobel and Canny Filters.
* ##### Thresholding.
* ##### Morphological Operations.

##### >> Notes  :
> - ##### To close image windows smoothly please **press Esc** on your keyboard, **don't close** it directly by clicking on 'X' to avoid kernel interruption.
> - ##### To run this lab using google colab follow below instructions:
> 1. Import `cv2_imshow` method using `from google.colab.patches import cv2_imshow`.
> 2. Replace all `cv.imshow()` with `cv2_imshow()` with one parameter **name of that image**.
> 3. Specify your path.

In [1]:
import cv2 as cv
import numpy as np

### Section 0, _Helper Functions_:
> ##### This cell contains some helper function such as error_handler, please run it without any modification.

In [2]:
# please don't modify this code.
def show_text_window(titles):
    black = np.zeros((len(titles) * 150, 400))
    for idx, t in enumerate(titles):
        place = idx + 1
        cv.putText(black, t, (10, place * 100),
                   cv.FONT_HERSHEY_SIMPLEX, 1, (200, 0, 200), 1, 2)
    cv.imshow("Values", black)


def get_updated_value(key, value, **kwargs):
    if key == ord('+'):
        return value, 0
    elif key == ord('-'):
        return -value, 0
    elif key == ord('*'):
        return 0, kwargs.get('value2', value)
    elif key == ord('/'):
        return 0, -kwargs.get('value2', value)
    elif key == 27:
        raise Exception('Terminated by user!')
    else:
        return 0, 0


def error_handler(func):
    def wrapper(*args, **kwargs):
        try:
            func(*args, **kwargs)
        except Exception as ex:
            cv.destroyAllWindows()
            print(f'{ex}')

    return wrapper


### Section 1, _Edge Detection_:
> #### In this section we will discover `Sobel` and `Canny` filters which are widely used for **edge detection** in computer vision applications.

#### 1.1 Sobel filter
> ##### It is particularly effective at highlighting abrupt changes in pixel intensity, which often correspond to edges in images.
> ##### The Sobel Operator combines Gaussian smoothing and differentiation.
![image info](https://i0.wp.com/theailearner.com/wp-content/uploads/2019/05/sobelxy.png?w=660&ssl=1)
![image info](https://i0.wp.com/theailearner.com/wp-content/uploads/2019/05/Sobx2.png?w=626&ssl=1)
* ##### [Click to check the documentation](https://docs.opencv.org/3.4/d2/d2c/tutorial_sobel_derivatives.html)

In [69]:
@error_handler
def sobel(path):
    image = cv.imread(path, 0)
    cv.imshow("Image", image)
    k = 1
    while True:
        sobel_x_filtered_image = cv.Sobel(image, cv.CV_64F, 1, 0, ksize=k)  # grad_x
        sobel_y_filtered_image = cv.Sobel(image, cv.CV_64F, 0, 1, ksize=k)  # grad_y

        grad = np.sqrt(sobel_x_filtered_image**2 + sobel_y_filtered_image**2)  # grad
        grad_norm = (grad * 255 / grad.max()).astype(np.uint8)  # grad_scaled

        sobel_x_filtered_image = cv.convertScaleAbs(sobel_x_filtered_image)  # grad_x_scaled
        sobel_y_filtered_image = cv.convertScaleAbs(sobel_y_filtered_image)  # grad_y_scaled

        cv.imshow("sobel_x_filtered_image", sobel_x_filtered_image)
        cv.imshow("sobel_y_filtered_image", sobel_y_filtered_image)
        cv.imshow("sobel_magnitude_filtered_image", grad_norm)

        show_text_window([f'Kernel Size {k, k}'])

        key = cv.waitKey(0)
        dk, _ = get_updated_value(key, 2)
        k += dk


path = "../Images/sudoku2.png"
sobel(path)


Terminated by user!


####  1.2 Canny Filter
##### >> it involves several steps:
> ##### Gaussian Smoothing -> Gradient Calculation -> Non-Maximum Suppression -> Double Thresholding -> Edge Tracking by Hysteresis.
* ##### [Click to check the documentation](https://docs.opencv.org/4.x/da/d22/tutorial_py_canny.html)
* ##### Click for more explanation: [Link1](https://medium.com/@rohit-krishna/coding-canny-edge-detection-algorithm-from-scratch-in-python-232e1fdceac7), [Link2](https://pyimagesearch.com/2021/05/12/opencv-edge-detection-cv2-canny/)



In [67]:
@error_handler
def canny(path):
    img = cv.imread(path, 0)
    cv.imshow('image_original', img)
    th1 = 100
    th2 = 200
    while True:
        filtered_image = cv.Canny(img, threshold1=th1, threshold2=th2)
        cv.imshow('filtered_image', filtered_image)

        show_text_window([f'Threshold1: {th1}', f'Threshold2: {th2}'])

        key = cv.waitKey(0)
        dt1, dt2 = get_updated_value(key, 10)
        th1 += dt1
        th2 += dt2

path = '../Images/home.jpg'
canny(path)

Terminated by user!


####  Sobel vs Canny
##### 1. The Sobel edge detection algorithm also finds the thickness of the edges.
##### 2. It is not a binary image, but a grayscale image.
##### 3. It also has many noises that we can reduce.

In [73]:
path = '../Images/messi.jpg'
sobel(path)

Terminated by user!


In [74]:
path = '../Images/messi.jpg'
canny(path)

Terminated by user!


### Section 2, _Thresholding_:
> ##### In this section we will discover the variant types of thresholding including binary, inverse binary, truncate and to zero.
![image info](https://media.geeksforgeeks.org/wp-content/uploads/20190505162413/Screenshot-4113.png)


In [43]:
@error_handler
def threshold(path):
    img = cv.imread(path, 0)
    rows, cols = img.shape    
    img = cv.resize(img, (cols // 2, rows // 2))
    cv.imshow('Image', img)


    thresh_types = [cv.THRESH_BINARY, cv.THRESH_BINARY_INV,
                    cv.THRESH_TRUNC, cv.THRESH_TOZERO, cv.THRESH_TOZERO_INV]

    thresh_names = ['THRESH_BINARY', 'THRESH_BINARY_INV',
                    'cv.THRESH_TRUNC', 'cv.THRESH_TOZERO', 'cv.THRESH_TOZERO_INV']

    initial_params = [(100, 255), (127, 255), (127, 0), (127, 0), (80, 0)]

    for thresh_type, thresh_name, params in zip(thresh_types, thresh_names, initial_params):
        threshold, max_val = params

        while True:
            ret, thresh = cv.threshold(img, threshold, max_val, thresh_type)
            show_text_window([f'Threshold: {threshold}', f'MaxVal: {max_val}'])
            cv.imshow(thresh_name, thresh)

            try:
                key = cv.waitKey(0)
                dt, dm = get_updated_value(key, 10)
                threshold += dt
                max_val += dm
            except:
                cv.destroyWindow(thresh_name)
                break

    cv.destroyAllWindows()


path = '../Images/numbers.png'
threshold(path)


### Section 3, _Morphological Operations_:
> #### Morphological operations are a set of image processing techniques in computer vision used for analyzing and manipulating the structure of objects within an image, In this section we will explore the variant types of them.

#### 3.1 Dilation
* ##### OR window filter.
* ##### It works by expands the boundaries of the foreground object in a binary image.
* ##### It's  useful for tasks like expanding and filling gaps in objects, joining nearby objects, and thickening object boundaries.

In [45]:
@error_handler
def dilation(path):
    img = cv.imread(path, 0)
    cv.imshow("Image", img)
    k, iterations = 5, 1

    while True:
        kernel = np.ones((k, k), np.uint8)
        dilation = cv.dilate(img, kernel, iterations=iterations)
        
        cv.imshow("dilation", dilation)
        show_text_window([f'Kernel Size {k}', f'Iterations {iterations}'])

        key = cv.waitKey(0)
        dk, di = get_updated_value(key, 2, value2=1)
        k += dk
        iterations += di


path = '../Images/circles.png'
dilation(path)


Terminated by user!


In [46]:
path = '../Images/blobs.png'
dilation(path)

Terminated by user!


### 3.2 Erosion
* ##### AND window filter.
* ##### It shrinks the boundaries of the foreground object in a binary image.
* ##### It's useful for tasks like removing noise, separating touching objects, and thinning object boundaries.

In [47]:
@error_handler
def erosion(path):
    img = cv.imread(path, 0)
    cv.imshow("Image", img)
    k, iterations = 5, 1
    while True:
        kernel = np.ones((k, k), np.uint8)
        erosion = cv.erode(img, kernel, iterations=iterations)
        
        cv.imshow("erosion", erosion)
        show_text_window([f'Kernel Size {k}', f'Iterations {iterations}'])

        key = cv.waitKey(0)
        dk, di = get_updated_value(key, 2, value2=1)
        k += dk
        iterations += di


path = '../Images/circles.png'
erosion(path)


Terminated by user!


In [48]:
path = '../Images/blobs.png'
erosion(path)

Terminated by user!


### 3.3 Opening
* ##### Opening is a sequence of erosion followed by dilation.
* ##### It's useful for removing small objects, noise, or fine details from the image.
* ##### Opening = Erosion + Dilation.

In [49]:
@error_handler
def opening(path):
    img = cv.imread(path, 0)
    cv.imshow("Image", img)
    k = 5
    while True:
        kernel = np.ones((k, k), np.uint8)
        erosion = cv.erode(img, kernel)
        opening = cv.morphologyEx(img, cv.MORPH_OPEN, kernel)

        cv.imshow("erosion", erosion)
        cv.imshow("opening", opening)
        show_text_window([f'Kernel Size {k}'])

        key = cv.waitKey(0)
        dk, _ = get_updated_value(key, 2)
        k += dk


path = '../Images/opening.png'
opening(path)


Terminated by user!


In [50]:
path = '../Images/blobs.png'
opening(path)

Terminated by user!


### 3.4 Closing
* ##### Closing is a sequence of dilation followed by erosion.
* ##### It's effective in closing small holes and gaps within objects.
* ##### Closing = Dilation + Erosion.

In [57]:
@error_handler
def closing(path):
    img = cv.imread(path, 0)
    cv.imshow("Image", img)
    
    k = 5
    while True:
        kernel = np.ones((k, k), np.uint8)
        dilation = cv.dilate(img, kernel)
        closing = cv.morphologyEx(img, cv.MORPH_CLOSE, kernel)
        
        cv.imshow("dilation", dilation)
        cv.imshow("closing", closing)
        show_text_window([f'Kernel Size {k}'])

        key = cv.waitKey(0)
        dk, _ = get_updated_value(key, 2)
        k += dk


path = '../Images/j.png'
closing(path)


Terminated by user!


In [59]:
path = '../Images/blobs.png'
closing(path)

Terminated by user!


### 3.5 Gradient
* ##### The morphological gradient is the difference between dilation and erosion of an image.
* ##### It highlights the boundaries of objects.
* ##### Gradient = Dilation - Erosion


In [62]:
@error_handler
def gradient(path):
    img = cv.imread(path, 0)
    cv.imshow("Image", img)
    k = 5
    while True:
        kernel = np.ones((k, k), np.uint8)
        gradient = cv.morphologyEx(img, cv.MORPH_GRADIENT, kernel)
        
        cv.imshow("gradient", gradient)
        show_text_window([f'Kernel Size {k}'])

        key = cv.waitKey(0)
        dk, _ = get_updated_value(key, 2)
        k += dk


path = '../Images/j.png'
gradient(path)

Terminated by user!


In [64]:
path = '../Images/blobs.png'
gradient(path)

Terminated by user!


## _The End_.